Funktion aus US1

In [6]:
import os
import pandas as pd

# Inputverzeichnis definieren
input_dir = '../data/raw_data_bb'

# Rekursive Dateisuche nach .tsv-Dateien
def find_tsv_files(directory):
    tsv_files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tsv'):
                full_path = os.path.join(root, file)
                tsv_files_list.append(full_path)
    return tsv_files_list

# Liste der .tsv-Dateien im Inputverzeichnis
tsv_files = find_tsv_files(input_dir)
print (f"Found {len(tsv_files)} .tsv files in {input_dir}")
for file in tsv_files:
    print(file)

Found 7 .tsv files in ../data/bold_binders
../data/bold_binders/leskinen_2016/Leskinen_full_raw_counts.tsv
../data/bold_binders/li_2020/Li_full_raw_counts.tsv
../data/bold_binders/guegler_2021/Guegler_T7_minusToxIN_full_raw_counts.tsv
../data/bold_binders/ceyssens_2014/Ceyssens_non-directional_full_raw_counts.tsv
../data/bold_binders/brandao_2021/Brandao_LB_full_raw_counts.tsv
../data/bold_binders/sprenger_2024/Sprenger_VC_delta_tdh_VP882_WT_full_raw_counts.tsv
../data/bold_binders/wolfram-schauerte_2022/Wolfram-Schauerte_full_raw_counts.tsv


### Extraktion von Gen-IDs aus GFF-Dateien

Die Funktion `extract_gene_ids_from_gff` dient dazu, relevante Gen-IDs aus einer GFF-Datei zu extrahieren und gleichzeitig die gesamte Datei als DataFrame für weitere Analysen bereitzustellen.
Zunächst wird die GFF-Datei eingelesen, wobei Kommentarzeilen (beginnend mit `#`) ignoriert werden. Anschließend wird jede gültige Zeile, die genau neun Spalten enthält, in ein `pandas` DataFrame umgewandelt.

Im nächsten Schritt filtert die Funktion gezielt nur jene Einträge heraus, deren Typfeld den Wert `"gene"` enthält. Aus dem Attribut-Feld der GFF-Datei werden dann zwei mögliche Bezeichner extrahiert:
- `ID=`: die interne eindeutige Kennung eines Gens
- `Name=`: ein möglicherweise alternativer oder gebräuchlicher Genname
Diese beiden Identifier werden kombiniert, um eine vollständige Menge (`set`) aller Gen-IDs zu erstellen. Zusätzlich gibt die Funktion die Anzahl der Geneinträge sowie das gesamte DataFrame zurück.

In [7]:
# Extract gene IDs from GFF file and return full DataFrame
def extract_gene_ids_from_gff(gff_file):
    with open(gff_file, 'r') as f:
        lines = [line for line in f if not line.startswith('#')]
    records = [line.strip().split('\t') for line in lines if len(line.strip().split('\t')) == 9]
    df = pd.DataFrame(records, columns=[
        'seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'
    ])
    gene_df = df[df['type'] == 'gene'].copy()
    gene_df['id_field'] = gene_df['attributes'].str.extract(r'ID=([^;]+)')
    gene_df['name_field'] = gene_df['attributes'].str.extract(r'Name=([^;]+)')
    combined_ids = pd.concat([gene_df['id_field'].dropna(), gene_df['name_field'].dropna()]).unique()
    return set(combined_ids), len(gene_df), df

### Vergleich von Count-Tabellen mit GFF-Dateien

Die Funktion `compare_with_multiple_gff_and_print_filtered` vergleicht eine `.tsv`-Datei mit mehreren GFF-Dateien in einem angegebenen Verzeichnis, um festzustellen, welche Gene aus der Tabelle in den GFF-Dateien enthalten sind.

Zu Beginn wird die `.tsv`-Datei eingelesen. Anschließend erfolgt eine Filterung: Es werden nur jene Zeilen berücksichtigt, bei denen das Feld `Entity` entweder leer ist oder den Eintrag `'host'` enthält – in der Regel handelt es sich dabei um "host"-Genes. Aus der ersten Spalte dieser gefilterten Tabelle werden die Gen-IDs extrahiert, die als Grundlage für den späteren Vergleich dienen.

Im nächsten Schritt durchläuft die Funktion alle `.gff`-Dateien, die sich im angegebenen Verzeichnis befinden. Für jede Datei werden zunächst mit Hilfe der Funktion `extract_gene_ids_from_gff` die darin enthaltenen Gen-IDs extrahiert. Anschließend wird ermittelt, welche dieser IDs mit den in der Zähltabelle enthaltenen übereinstimmen (matched) und welche nicht (unmatched). Auf dieser Basis wird die Übereinstimmungsrate zwischen Zähltabelle und GFF-Datei berechnet und ausgegeben.

Sollte diese Übereinstimmungsrate unter 100 % liegen, wird zusätzlich überprüft, ob sich die nicht gefundenen Gen-IDs eventuell in anderen Attributfeldern der jeweiligen GFF-Datei befinden. Solche Fälle werden erfasst und dokumentiert, um nachzuweisen, ob die Geninformationen möglicherweise an unerwarteter Stelle enthalten sind.

Im Anschluss werden sämtliche als „unmatched“ identifizierten Gene aus der gefilterten Zähltabelle entfernt. Auf Basis dieser bereinigten Tabelle wird die Übereinstimmungsrate erneut berechnet, um die Qualität der bereinigten Daten zu bewerten.

Falls der Parameter `save_filtered=True` gesetzt wurde, speichert die Funktion die bereinigte Zähltabelle unter dem Namen `*_filtered.tsv` im gleichen Verzeichnis wie die Originaldatei.

In [8]:
def compare_with_multiple_gff_and_print_filtered(count_table_path, gff_folder_path, save_filtered=False):
    counts = pd.read_csv(count_table_path, sep='\t')
    filtered = counts[(counts['Entity'].fillna('') == '') | (counts['Entity'] == 'host')]
    filtered_gene_ids = set(filtered.iloc[:, 0])

    print(f"\nTotal 'host'/empty genes in count table: {len(filtered_gene_ids)}\n")

    for file in os.listdir(gff_folder_path):
        if file.endswith('.gff'):
            gff_path = os.path.join(gff_folder_path, file)
            gff_gene_ids, gff_total_genes, gff_df = extract_gene_ids_from_gff(gff_path)

            matched = filtered_gene_ids & gff_gene_ids
            unmatched = filtered_gene_ids - gff_gene_ids
            match_rate = len(matched) / len(filtered_gene_ids) * 100 if filtered_gene_ids else 0

            if matched:
                print(f"File: {file}")
                print(f"   Genes in GFF:               {gff_total_genes}")
                print(f"   Matched genes:               {len(matched)}")
                print(f"   Unmatched genes:             {len(unmatched)}")
                print(f"   Match rate (before removal): {match_rate:.2f}%")

                if match_rate < 100.0:
                    # Search unmatched genes in other GFF feature types
                    search_results = []
                    for gene_id in unmatched:
                        found = gff_df['attributes'].str.contains(gene_id, na=False)
                        matches = gff_df[found]
                        search_results.append({
                            'Unmatched Gene ID': gene_id,
                            'Found in any attribute': not matches.empty,
                            'Example attribute match': matches.iloc[0]['attributes'] if not matches.empty else ""
                        })
                    found_count = sum(1 for r in search_results if r['Found in any attribute'])
                    print(f"   Found in other feature types: {found_count}")
                    print(f"   First {len(unmatched)} unmatched gene IDs: {list(unmatched)[:10]}")

                    # Remove unmatched genes from the filtered DataFrame
                    before_count = len(filtered)
                    filtered = filtered[~filtered.iloc[:, 0].isin(unmatched)]
                    after_count = len(filtered)
                    print(f"   {before_count - after_count} unmatched genes removed from count table")

                    # Recalculate match rate after removal
                    new_gene_ids = set(filtered.iloc[:, 0])
                    new_matched = new_gene_ids & gff_gene_ids
                    new_match_rate = len(new_matched) / len(new_gene_ids) * 100 if new_gene_ids else 0
                    print(f"   Match rate after removal: {new_match_rate:.2f}%\n")

    if save_filtered:
        out_path = count_table_path.replace(".tsv", "_filtered.tsv")
        filtered.to_csv(out_path, sep='\t', index=False)
        print(f"\nFiltered count table saved to: {out_path}")


### Automatisierte Verarbeitung von Count-Tabellen und GFF-Dateien

Die Funktion `run_all` führt den Schritt für den Vergleich automatisiert aus:

**Vergleich**: Alle `.tsv`-Count-Tabellen werden mit den konvertierten `.gff`-Dateien abgeglichen, um Gen-Übereinstimmungen zu prüfen.

So lassen sich mehrere Datensätze effizient in einem Durchlauf analysieren.


In [9]:
# Apply analysis to all count tables in a folder
def run_all(tsv_files, gff_folder_path):
    print("Comparing count tables to GFF files")
    for count_file_path in tsv_files:
        print(f"\nProcessing count table: {os.path.basename(count_file_path)}")
        compare_with_multiple_gff_and_print_filtered(count_file_path, gff_folder_path)

Bei den Ergebnissen fällt auf, dass einige Datensätze keine vollständige Übereinstimmung (100 %) mit den jeweiligen GFF-Dateien aufweisen. Die betroffenen Gene sollten genauer betrachtet werden.

In [10]:
#Define host_gff directory
host_gff_dir = '../data/raw_data_bb/host_gff_files/'

run_all(tsv_files, host_gff_dir)

Comparing count tables to GFF files

Processing count table: Leskinen_full_raw_counts.tsv

Total 'host'/empty genes in count table: 3911

File: new_Yersinia_host_genome.gff
   Genes in GFF:               4003
   Matched genes:               3911
   Unmatched genes:             0
   Match rate (before removal): 100.00%

Processing count table: Li_full_raw_counts.tsv

Total 'host'/empty genes in count table: 3694

File: new_Clostridioides_difficile_host_genomic.gff
   Genes in GFF:               3732
   Matched genes:               3694
   Unmatched genes:             0
   Match rate (before removal): 100.00%

Processing count table: Guegler_T7_minusToxIN_full_raw_counts.tsv

Total 'host'/empty genes in count table: 4484

File: Enterobacteria_host.genomic.gff
   Genes in GFF:               4494
   Matched genes:               4472
   Unmatched genes:             12
   Match rate (before removal): 99.73%
   Found in other feature types: 0
   First 12 unmatched gene IDs: ['gene-b4838', 'ge